In [ ]:
!mkdir "/tmp/Patches"
!mkdir "/tmp/Patches/Luc Van Tien"
!mkdir "/tmp/Patches/Tale of Kieu 1866"
!mkdir "/tmp/Patches/Tale of Kieu 1871"
!mkdir "/tmp/Patches/Tale of Kieu 1872"
!mkdir "/tmp/Patches/DVSKTT-1 Quyen thu"
!mkdir "/tmp/Patches/DVSKTT-2 Ngoai ky toan thu"
!mkdir "/tmp/Patches/DVSKTT-3 Ban ky toan thu"
!mkdir "/tmp/Patches/DVSKTT-4 Ban ky thuc luc"
!mkdir "/tmp/Patches/DVSKTT-5 Ban ky tuc bien"

# Import patches

In [ ]:
import glob
PATH_LABELS = glob.glob("/tmp/crop/Patches/*/Transcription.txt")
PATH_SOURCE = '/tmp/crop/Patches'
PATH_TARGET = '/tmp/Patches'

In [ ]:
def read_labels(path):
    path_file = re.findall(r'Patches\/(.*)?\/', path)[0]
    with open(path, 'r', encoding='utf-8') as f:

In [ ]:
dataset = []
for path in PATH_LABELS: 
    dataset.extend(read_labels(path))
max([len(x[1]) for x in dataset])

24

# Split Text Recognition patches

### Split patches into train & val DataFrames using IHR-NomDB R-Score

In [ ]:
from IHRNomDB_Rs import calculate_r_scores
r_scores = calculate_r_scores(dataset)
r_scores_sorted = sorted(r_scores, key=lambda x: x[2], reverse=True)
r_scores_sorted[0]

['DVSKTT-4 Ban ky thuc luc/DVSKTT_ban_thuc_XIV_53a_4.jpg',
 '門公以衛士不義遂殺之十二月初一日帝飲',
 1449896006]

In [ ]:
import numpy as np
import pandas as pd

df = pd.DataFrame(data=np.array(r_scores_sorted)[:, :2], columns=['path', 'text'])
df = df[df['path'] != 'DVSKTT-5 Ban ky tuc bien/DVSKTT_ban_tuc_XIX_23a_7.jpg']
df_train = df.sample(frac=0.8)
df_val = df.drop(df_train.index)
len(df)

38318


In [ ]:
from IHRNomDB_Rs import print_intersection
print_intersection(df_val['text'].tolist(), df_train['text'].tolist())

Characters intersection train 93.2405165456013
Characters intersection val 64.41315862838026


### Apply splitting on the real data based on above DataFrames

In [ ]:
import os

label_train = [f'{path}\t{text}' for path, text in df_train.values]
with open(os.path.join(PATH_TARGET, 'Train.txt'), 'w+', encoding='utf-8') as f:
    f.write('\n'.join(label_train))

label_val = [f'{path}\t{text}' for path, text in df_val.values]
with open(os.path.join(PATH_TARGET, 'Validate.txt'), 'w+', encoding='utf-8') as f:
    f.write('\n'.join(label_val))

In [ ]:
import shutil
from tqdm.notebook import tqdm

def split_patches(labels):
    for elm in tqdm(labels):
        path = elm.split('\t')[0]
        source = os.path.join(PATH_SOURCE, path)
        source = source.replace('crop_img/', '')

        target = path.replace('crop_img/', '')
        target = target.replace('_crop', '')
        target = os.path.join(PATH_TARGET, target)
        shutil.copy(source, target)

In [ ]:
split_patches(df_train['path'])
split_patches(df_val['path'])